In [3]:
# --------------------------
# 1️⃣ Imports
# --------------------------
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random

# --------------------------
# 2️⃣ Seed for reproducibility
# --------------------------
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything()

# --------------------------
# 3️⃣ Device selection
# --------------------------
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

# --------------------------
# 4️⃣ Load data
# --------------------------
df = pd.read_csv("frames_errors.csv", header=None)
df.columns = [
    "block_id","frame_idx","E_mu_Z","E_mu_phys_est","E_mu_X","E_nu1_X","E_nu2_X",
    "E_nu1_Z","E_nu2_Z","N_mu_X","M_mu_XX","M_mu_XZ","M_mu_X","N_mu_Z","M_mu_ZZ",
    "M_mu_Z","N_nu1_X","M_nu1_XX","M_nu1_XZ","M_nu1_X","N_nu1_Z","M_nu1_ZZ","M_nu1_Z",
    "N_nu2_X","M_nu2_XX","M_nu2_XZ","M_nu2_X","N_nu2_Z","M_nu2_ZZ","M_nu2_Z","nTot",
    "bayesImVoltage","opticalPower","polarizerVoltages[0]","polarizerVoltages[1]",
    "polarizerVoltages[2]","polarizerVoltages[3]","temp_1","biasVoltage_1","temp_2",
    "biasVoltage_2","synErr","N_EC_rounds","maintenance_flag","estimator_name",
    "f_EC","E_mu_Z_est","R","s","p"
]

# Drop columns not used
df_base = df.drop(["E_mu_phys_est", "f_EC", "estimator_name"], axis=1)

# --------------------------
# 5️⃣ Normalize features and targets
# --------------------------
features = df_base.drop(columns=["R", "s", "p", "block_id", "frame_idx"]).values
targets = df_base[["R", "s", "p"]].values

# Feature scaling
scaler_X = StandardScaler()
features_scaled = scaler_X.fit_transform(features)

# Target scaling (MinMax for [0,1])
scaler_y = MinMaxScaler()
targets_scaled = scaler_y.fit_transform(targets)

# Add series id and frame_idx back
series_ids = df_base["block_id"].values
frame_idx = df_base["frame_idx"].values

# --------------------------
# 6️⃣ Time series dataset class
# --------------------------
HISTORY = 300
HORIZON = 8

class TimeSeriesDataset(Dataset):
    def __init__(self, features, targets, series_ids, history=HISTORY, horizon=HORIZON):
        self.X, self.y, self.series = [], [], []
        self.history = history
        self.horizon = horizon
        unique_ids = np.unique(series_ids)
        for uid in unique_ids:
            idx = np.where(series_ids == uid)[0]
            for i in range(len(idx) - history - horizon):
                self.X.append(features[idx[i:i+history]])
                self.y.append(targets[idx[i+history:i+history+horizon]])
                self.series.append(uid)
        self.X = np.array(self.X, dtype=np.float32)
        self.y = np.array(self.y, dtype=np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Split train/val/test 70/15/15
unique_ids = np.unique(series_ids)
n_train = int(len(unique_ids)*0.7)
n_val = int(len(unique_ids)*0.15)

train_ids = unique_ids[:n_train]
val_ids = unique_ids[n_train:n_train+n_val]
test_ids = unique_ids[n_train+n_val:]

train_mask = np.isin(series_ids, train_ids)
val_mask = np.isin(series_ids, val_ids)
test_mask = np.isin(series_ids, test_ids)

train_dataset = TimeSeriesDataset(features_scaled[train_mask], targets_scaled[train_mask], series_ids[train_mask])
val_dataset = TimeSeriesDataset(features_scaled[val_mask], targets_scaled[val_mask], series_ids[val_mask])
test_dataset = TimeSeriesDataset(features_scaled[test_mask], targets_scaled[test_mask], series_ids[test_mask])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# --------------------------
# 7️⃣ FEDformer model (simplified)
# --------------------------
class FourierBlock(nn.Module):
    def __init__(self, in_channels, out_channels, seq_len):
        super(FourierBlock, self).__init__()
        self.weights = nn.Parameter(torch.randn(in_channels, out_channels, dtype=torch.cfloat))
        self.seq_len = seq_len

    def forward(self, x):
        x_ft = torch.fft.rfft(x, dim=1)
        out_ft = torch.einsum("bsi,io->bso", x_ft, self.weights)
        x = torch.fft.irfft(out_ft, n=self.seq_len, dim=1)
        return x

class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_ff, seq_len):
        super(EncoderLayer, self).__init__()
        self.attn = FourierBlock(d_model, d_model, seq_len)
        self.norm1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(nn.Linear(d_model, d_ff), nn.GELU(), nn.Linear(d_ff, d_model))
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.norm1(x + self.attn(x))
        x = self.norm2(x + self.ff(x))
        return x

class FEDformer(nn.Module):
    def __init__(self, seq_len, pred_len, d_model=64, d_ff=128, e_layers=2, enc_in=features.shape[1], c_out=3):
        super(FEDformer, self).__init__()
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.enc_embedding = nn.Linear(enc_in, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, d_ff, seq_len) for _ in range(e_layers)])
        self.projection = nn.Linear(d_model, c_out)

    def forward(self, x_enc):
        x = self.enc_embedding(x_enc)
        for layer in self.encoder:
            x = layer(x)
        out = self.projection(x[:, -self.pred_len:, :])
        return out

# --------------------------
# 8️⃣ Train loop
# --------------------------
def train_model(model, train_loader, val_loader, n_epochs=5, lr=1e-3, patience=2):
    model = model.to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_val = float('inf')
    counter = 0

    for epoch in range(n_epochs):
        model.train()
        train_loss = 0
        for X, y in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            y_pred = model(X)
            loss = criterion(y_pred, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.size(0)

        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE), y.to(DEVICE)
                y_pred = model(X)
                val_loss += criterion(y_pred, y).item() * X.size(0)
        val_loss /= len(val_loader.dataset)

        print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")

        # Early stopping
        if val_loss < best_val:
            best_val = val_loss
            counter = 0
            torch.save(model.state_dict(), "fedformer_best.pt")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered")
                break

    model.load_state_dict(torch.load("fedformer_best.pt"))
    return model

# --------------------------
# 9️⃣ Train
# --------------------------
model = FEDformer(seq_len=HISTORY, pred_len=HORIZON)
model = train_model(model, train_loader, val_loader, n_epochs=20, lr=1e-3)

# --------------------------
# 10️⃣ Predict & inverse transform
# --------------------------
model.eval()
all_preds = []
with torch.no_grad():
    for X, y in test_loader:
        X = X.to(DEVICE)
        preds = model(X)
        all_preds.append(preds.cpu().numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_preds_inv = scaler_y.inverse_transform(all_preds.reshape(-1,3)).reshape(all_preds.shape)

print("Predictions shape:", all_preds_inv.shape)
print(all_preds_inv)


Using device: cuda
Epoch 1: Train Loss=0.0398, Val Loss=0.0366
Epoch 2: Train Loss=0.0342, Val Loss=0.0357
Epoch 3: Train Loss=0.0329, Val Loss=0.0346
Epoch 4: Train Loss=0.0319, Val Loss=0.0345
Epoch 5: Train Loss=0.0310, Val Loss=0.0350
Epoch 6: Train Loss=0.0298, Val Loss=0.0348
Early stopping triggered
Predictions shape: (11322, 8, 3)
[[[7.9696274e-01 3.4359531e+03 1.3317471e+03]
  [7.8824240e-01 3.4219531e+03 1.3751705e+03]
  [7.9714757e-01 3.3065188e+03 1.4430303e+03]
  ...
  [7.9191023e-01 3.3162703e+03 1.4249875e+03]
  [7.9928583e-01 3.4382871e+03 1.2764768e+03]
  [8.0560917e-01 3.3522676e+03 1.4436274e+03]]

 [[7.8795505e-01 3.4276555e+03 1.3695000e+03]
  [7.9685938e-01 3.3057419e+03 1.4452045e+03]
  [7.9711235e-01 3.4107690e+03 1.3280752e+03]
  ...
  [7.9999328e-01 3.4567134e+03 1.2601501e+03]
  [8.0300480e-01 3.3362415e+03 1.4570479e+03]
  [8.0894941e-01 3.4996064e+03 1.2608839e+03]]

 [[7.9820353e-01 3.3210222e+03 1.4311028e+03]
  [7.9743361e-01 3.4094126e+03 1.3285214e+03]

In [ ]:
print(all_preds_inv)